In [ ]:
!pip install transformers torch
!pip install accelerate
!pip install sentencepiece # Some models might require this

In [ ]:
from google.colab import userdata
import os

os.environ["NL_TOKEN"] = userdata.get('NL_TOKEN')

In [ ]:
db_schema = """
CREATE TABLE student (
    student_id INTEGER,
    name VARCHAR,
    marks INTEGER,
    grade VARCHAR,
    age INTEGER
);
"""
from transformers import pipeline
model_name = "gaussalgo/T5-LM-Large-text2sql-spider"
text_to_sql_pipeline = pipeline("text2text-generation", model=model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# Install pandasql
!pip install pandasql
# Install pandasql (If it hasn't been installed in the environment)
# !pip install pandasql

import pandas as pd
import numpy as np
from pandasql import sqldf

# Set a seed for reproducibility
np.random.seed(42)

# Define the number of students
num_students = 30

# Create a DataFrame named 'student' to match the schema, now with 30 rows
student = pd.DataFrame({
    'student_id': np.arange(1, num_students + 1),  # Generates IDs from 1 to 30
    'name': [f'Student_{i}' for i in range(1, num_students + 1)],
    'marks': np.random.randint(50, 100, num_students),  # 30 random marks
    'grade': np.random.choice(['A', 'B', 'C', 'D'], num_students),  # 30 random grades
    'age': np.random.randint(18, 25, num_students)  # 30 random ages
})

# Display the first few rows (will show 5 rows by default)
print("Dummy Student DataFrame (30 rows):")
print(student.head())
# ---
# Define a function to execute SQL on the DataFrame
def psql(q):
    return sqldf(q, globals())

# Example usage of psql function to verify the count
# print("\nSQL verification of row count:")
# print(psql("SELECT COUNT(*) FROM student;"))

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=ca29b1529e8dd398073102c079f4ce21e6b1103c977e1195a3eda4826d6913ed
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql
Dummy Student DataFrame (30 rows):
   student_id       name  marks grade  age
0           1  Student_1     88     C   23
1           2  Student_2     78     C   19
2           3  Student_3     64     C   19
3           4  Student_4     92     B   18
4           5  Student_5     57     D   19


In [ ]:
# Set Pandas options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Print the entire student DataFrame
print(student)

    student_id        name  marks grade  age
0            1   Student_1     88     C   23
1            2   Student_2     78     C   19
2            3   Student_3     64     C   19
3            4   Student_4     92     B   18
4            5   Student_5     57     D   19
5            6   Student_6     70     D   22
6            7   Student_7     88     D   19
7            8   Student_8     68     D   21
8            9   Student_9     72     C   21
9           10  Student_10     60     B   24
10          11  Student_11     60     B   21
11          12  Student_12     73     C   24
12          13  Student_13     85     B   21
13          14  Student_14     89     C   22
14          15  Student_15     73     D   24
15          16  Student_16     52     C   20
16          17  Student_17     71     D   23
17          18  Student_18     51     D   18
18          19  Student_19     73     A   21
19          20  Student_20     93     C   19
20          21  Student_21     79     A   21
21        

In [ ]:
# V1
nl_query_one = "Show all students with marks above 80."
prompt_one = f"### {db_schema}\n### Task\n{nl_query_one}"
sql_one = text_to_sql_pipeline(prompt_one)

print(f"Query 1 Natural Language: {nl_query_one}")
print(f"Query 1 Generated SQL: {sql_one[0]['generated_text']}")

# Execute the generated SQL on the dummy data
result_one = psql(sql_one[0]['generated_text'])
print("Query 1 Result on Dummy Data:")
print(result_one)

Query 1 Natural Language: Show all students with marks above 80.
Query 1 Generated SQL: SELECT name FROM student WHERE marks > 80
Query 1 Result on Dummy Data:
         name
0   Student_1
1   Student_4
2   Student_7
3  Student_13
4  Student_14
5  Student_20
6  Student_22
7  Student_25
8  Student_28
9  Student_30


In [ ]:
# QUERY_V2
# The natural language query is changed to be more explicit.
nl_query_two = "Show the name and marks and age of students with age greater than 20 and grade 'A'."

# The prompt now explicitly tells the model to use the "student" table without a JOIN.
prompt_two = f"### {db_schema}\n### Task\n{nl_query_two}"
sql_two = text_to_sql_pipeline(prompt_two)

print(f"Query 2 Natural Language: {nl_query_two}")
print(f"Query 2 Generated SQL: {sql_two[0]['generated_text']}")

# Execute the generated SQL on the dummy data
result_two = psql(sql_two[0]['generated_text'])
print("Query 2 Result on Dummy Data:")
print(result_two)

Query 2 Natural Language: Show the name and marks and age of students with age greater than 20 and grade 'A'.
Query 2 Generated SQL: SELECT name, marks, age FROM student WHERE age > 20 AND grade = 'A'
Query 2 Result on Dummy Data:
         name  marks  age
0  Student_19     73   21
1  Student_21     79   21
2  Student_24     70   23
3  Student_25     82   23
4  Student_29     74   22


In [ ]:
nl_query_v3 = "Find the average marks of students."
prompt_v3 = f"### {db_schema}\n### Task\n{nl_query_v3}"
sql_v3 = text_to_sql_pipeline(prompt_v3)

print(f"V3 Natural Language: {nl_query_v3}")
print(f"V3 Generated SQL: {sql_v3[0]['generated_text']}")

# Execute the generated SQL
result_v3 = psql(sql_v3[0]['generated_text'])
print("V3 Query Result on Dummy Data:")
print(result_v3)

V3 Natural Language: Find the average marks of students.
V3 Generated SQL: SELECT avg(marks) FROM student
V3 Query Result on Dummy Data:
   avg(marks)
0        74.1


In [ ]:
#QUERY_V4
# V4: Ordering Query
nl_query_four = "Show students ordered by marks descending."
prompt_four = f"### {db_schema}\n### Task\n{nl_query_four}"
sql_four = text_to_sql_pipeline(prompt_four)

print(f"V4 Natural Language: {nl_query_four}")
print(f"V4 Generated SQL: {sql_four[0]['generated_text']}")

# Execute the generated SQL
result_four = psql(sql_four[0]['generated_text'])
print("V4 Query Result on Dummy Data:")
print(result_four)

V4 Natural Language: Show students ordered by marks descending.
V4 Generated SQL: SELECT name FROM student ORDER BY marks DESC
V4 Query Result on Dummy Data:
          name
0   Student_30
1   Student_20
2   Student_28
3    Student_4
4   Student_14
5    Student_1
6    Student_7
7   Student_22
8   Student_13
9   Student_25
10  Student_21
11   Student_2
12  Student_29
13  Student_12
14  Student_15
15  Student_19
16   Student_9
17  Student_17
18  Student_27
19   Student_6
20  Student_24
21   Student_8
22   Student_3
23  Student_26
24  Student_10
25  Student_11
26   Student_5
27  Student_16
28  Student_18
29  Student_23


In [ ]:
#QUERY_V5

# ---  Average of ALL Marks ---
# Natural Language Query
nl_query_five = "Find the average mark from all students."

# Define the schema (reusing the placeholder)
schema_description = "Table student (student_id INT, name TEXT, marks INT, grade TEXT, age INT)"
db_schema = schema_description # Keeping this for the prompt structure

# Create the full prompt for the text-to-SQL pipeline
prompt_five = f"### {db_schema}\n### Task\n{nl_query_five}"

# Manually generate the expected SQL for execution
# The SQL equivalent is using AVG(marks) without GROUP BY
generated_sql_text_five = "SELECT AVG(marks) FROM student;"

print(f"\n--- Fifth Query Execution (Average of ALL Students) ---")
print(f"NL Query: {nl_query_five}")
print(f"Generated SQL: {generated_sql_text_five}")

# Execute the generated SQL on the dummy data
result_five = psql(generated_sql_text_five)
print("Query Result on Dummy Data:")
print(result_five)


--- Fifth Query Execution (Average of ALL Students) ---
NL Query: Find the average mark from all students.
Generated SQL: SELECT AVG(marks) FROM student;
Query Result on Dummy Data:
   AVG(marks)
0        74.1


In [ ]:


#  Aggregation and Grouping Query
nl_query = "Show average marks from student by grade."
prompt_five = f"### {db_schema}\n### Task\n{nl_query}"
sql_five = text_to_sql_pipeline(prompt_five)

print(f"V5 Natural Language: {nl_query}")
print(f"V5 Generated SQL: {sql_five[0]['generated_text']}")

# Execute the generated SQL
result_five = psql(sql_five[0]['generated_text'])
print("V5 Query Result on Dummy Data:")
print(result_five)

V5 Natural Language: Show average marks from student by grade.
V5 Generated SQL: SELECT avg(marks) FROM student GROUP BY grade
V5 Query Result on Dummy Data:
   avg(marks)
0   75.600000
1   73.600000
2   73.454545
3   74.333333


In [ ]:
#This query uses a simple COUNT function to find the total number of students.
nl_query_count = "How many students are there?"
prompt_count = f"### {db_schema}\n### Task\n{nl_query_count}"
sql_count = text_to_sql_pipeline(prompt_count)

print(f"Count Query: {nl_query_count}")
print(f"Generated SQL: {sql_count[0]['generated_text']}")

# Execute the generated SQL
result_count = psql(sql_count[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result_count)

Count Query: How many students are there?
Generated SQL: SELECT count(*) FROM student
Query Result on Dummy Data:
   count(*)
0        30


In [ ]:
nl_query = "Count the number of students for each grade."
prompt = f"### {db_schema}\n### Task\n{nl_query}"
sql = text_to_sql_pipeline(prompt)
print(f"Generated SQL: {sql[0]['generated_text']}")
result = psql(sql[0]['generated_text'])
print("Result:")
print(result)

Generated SQL: SELECT count(*) , grade FROM student GROUP BY grade
Result:
   count(*) grade
0         5     A
1         5     B
2        11     C
3         9     D


In [ ]:
#V6
nl_query_six = "Show the names of students with marks above 85 and an age less than 22."
prompt_six = f"### {db_schema}\n### Task\n{nl_query_six}"
sql_six = text_to_sql_pipeline(prompt_six)

print(f"V6 Natural Language: {nl_query_six}")
print(f"V6 Generated SQL: {sql_six[0]['generated_text']}")

# Manually correct the generated SQL
corrected_sql_six = "SELECT name FROM student WHERE marks > 85 AND age < 22"
print(f"V6 Corrected SQL: {corrected_sql_six}")

result_six = psql(corrected_sql_six)
print("V6 Query Result on Dummy Data:")
print(result_six)

V6 Natural Language: Show the names of students with marks above 85 and an age less than 22.
V6 Generated SQL: SELECT name FROM student WHERE marks > 85 AND age  22
V6 Corrected SQL: SELECT name FROM student WHERE marks > 85 AND age < 22
V6 Query Result on Dummy Data:
         name
0   Student_4
1   Student_7
2  Student_20
3  Student_22
